In [147]:
import numpy as np
import math
from scipy import linalg, sparse

In [149]:
def covnw(data, demean=True, *lag):
    """
    Long-run covariance estimation using Newey-West (Bartlett) weights
    Usage:
        [V] = conw(data)
        [V] = conw(data,demean,lag)
    Inputs:
        data   - matrix TxK of data
        nlag   - non-negative integer containing the lag length to use. If empty or not included,
            nlag = min(mth.floor(1.2*T^(1/3)),T) is used
        demean - boolean value (true and false) indicating whether the mean should be subtracted when computing the covariance
    Outputs:
        V - A K by K covariance matrix estimated using Newey-West (Bartlett) weights
        
        EXAMPLES:
      Simulate an AR(1)
          y = armaxfilter_simulate(1000,0,1,.9);
      Newey-West covariance with automatic BW selection
          lrcov = covnw(y)
      Newey-West covariance with 10 lags
          lrcov = covnw(y, 10)
      Newey-West covariance with 10 lags and no demeaning
          lrcov = covnw(y, False, 10)
    """
    T = data.shape[0]
    if(not lag):
        nlag = min(math.floor(1.2*T**(1/3)),T)
    else:
        nlag = lag[0]
    if(math.floor(nlag)!=nlag or nlag<0):
            raise Exception("NLAG must be a non-negative integer.")
    if(not isinstance(demean, bool)):
        raise Exception("DEMEAN must be either logical True or False.")
    if(demean):
        data = data - np.tile(np.mean(data,axis=0),(T,1))
    #NW weights
    w = ((nlag+1)*np.matrix(np.ones(nlag+1,))-np.mat(np.arange(0,nlag+1,1)))/(nlag+1)
    #Start of covariance
    V = np.dot(data.T,data)/T
    for i in range(1,nlag+1):
        Gammai = (data[i:T,:].T.dot(data[0:T-i,:]))/T
        GplusGprime = (Gammai + Gammai.T)
        V += w.item(0,i)*GplusGprime
    return V